In [1]:
import numpy as np
import tensorflow as tf
import json
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Embedding
from CocktailEmbeddingMaker import CocktailEmbeddingMaker
from CocktailEmbeddingMaker import Eval
import wandb
# 데이터 로드
with open('./train_data.json', 'r') as f:
    json_data = json.load(f)
with open('../flavor.json', 'r') as f:
    flavor_data = json.load(f)
test_user_list = []

attributes = ['ABV', 'boozy', 'sweet', 'sour', 'bitter', 'umami', 'salty', 'astringent', 'Perceived_temperature', 'spicy', 'herbal', 'floral', 'fruity', 'nutty', 'creamy', 'smoky']
user_num = 5
for i in range(user_num):
    user = {}
    user['user_id'] = i
    preference = {}
    user['ABV'] = np.random.randint(0,60)
    for attribute in attributes[2:]:
        user[attribute] = np.random.randint(0,100)
    test_user_list.append(user)

# Random user_list 생성 및 저장
def generate_random_user_list(num_users):
    user_list = []
    attributes = ['ABV', 'boozy', 'sweet', 'sour', 'bitter', 'umami', 'salty', 'astringent', 'Perceived_temperature', 'spicy', 'herbal', 'floral', 'fruity', 'nutty', 'creamy', 'smoky']

    for i in range(num_users):
        user = {
            'user_id': i,
            'ABV': np.random.randint(0, 60),
        }
        for attribute in attributes[2:]:
            user[attribute] = np.random.randint(0, 100)
        user_list.append(user)

    with open(f'user_list_v1_{num_users}.json', 'w') as f:
        json.dump(user_list, f)

    print("Random user_list generated and saved.")


class RecipeGenerationModel:
    #RecipeGenerationModel(cocktail_embedding_maker, wandb_flag=True, max_recipe_length=10)
    def __init__(self, cocktail_embedding_maker,wandb_Flag=False, max_recipe_length=10):
        self.cocktail_embedding_maker = cocktail_embedding_maker
        self.ingredient_ids = cocktail_embedding_maker.ingredient_ids
        self.num_ingredients = cocktail_embedding_maker.num_ingredients
        self.max_recipe_length = max_recipe_length
        self.ingredient_embedding_matrix = cocktail_embedding_maker.create_ingredient_embedding_matrix()
        self.sweep_config = None
        self.evaluation_metrics=None
        self.sweep_id = None
        self.wandb = wandb_Flag
        self.model = self.build_model()
        self.total_amount = 200
        self.Eval = Eval(json_data,flavor_data,self.total_amount)
        self.attributes = ['ABV', 'boozy', 'sweet', 'sour', 'bitter', 'umami', 'salty', 'astringent', 'Perceived_temperature', 'spicy', 'herbal', 'floral', 'fruity', 'nutty', 'creamy', 'smoky']
        self.evaluation_metrics = ['diversity', 'abv_match', 'taste_match']
        # Best 모델 판정 및 저장
    def save_best_model(self, performance, abv_match, taste_match, threshold_performance, threshold_abv_match, threshold_taste_match,run_id):

        if performance >= threshold_performance and abv_match >= threshold_abv_match and taste_match >= threshold_taste_match:
            self.model.model.save(f'best_model_{run_id}.h5')
            print("Best model saved.")
        else:
            print("Model does not meet the threshold criteria.")

    def build_model(self):
        model = Sequential([
            Embedding(self.num_ingredients, self.ingredient_embedding_matrix.shape[1],
                      weights=[self.ingredient_embedding_matrix], input_length=self.max_recipe_length, trainable=False),
            LSTM(128, return_sequences=True),
            LSTM(128),
            Dense(64, activation='gelu'),
            Dense(self.num_ingredients, activation='softmax')
        ])
        model.compile(loss='categorical_crossentropy', optimizer='adam',metrics=['accuracy'])
        return model

    def train(self, recipes, epochs=50, batch_size=32,learning_rate=0.001):
        ingredient_sequences = []
        next_ingredients = []

        for recipe in recipes:
            sequence = [self.ingredient_ids[self.cocktail_embedding_maker.normalize_string(ingredient)] for ingredient in recipe]
            for i in range(1, len(sequence)):
                ingredient_sequences.append(sequence[:i])
                next_ingredients.append(sequence[i])

        ingredient_sequences = tf.keras.preprocessing.sequence.pad_sequences(ingredient_sequences, maxlen=self.max_recipe_length)
        next_ingredients = tf.keras.utils.to_categorical(next_ingredients, num_classes=self.num_ingredients)
        evaluation_interval = 5
        if self.wandb:
            # wandb 초기화
            wandb.init(project='cocktail_recipe_generation')


        # for epoch in range(epochs):
        history = self.model.fit(ingredient_sequences, next_ingredients, epochs=epochs, batch_size=batch_size, verbose=0)
        loss = history.history['loss'][0]
        accuracy = history.history['accuracy'][0]
        if self.wandb:
            # wandb 로깅
            wandb.log({
                'epoch': epochs,
                'loss': loss,
                'accuracy': accuracy,
            })
        # eval을 호출 해서 평가를 수행하고 결과를 evaluation_result에 저장한다. 
        # 저장후 evaluation_metrics에 지정된 평가 지표 합을 계산해서 performance변수에 저장한다. 
        #wandb에 performance와 개별 평가 지표결과를 로깅한다. 
            # 모델 평가
        # print("evaluating model")

        evaluation_results,recipe_profile_list = self.Eval.evaluate_model(self.model, test_user_list)
        if self.wandb:
            for recipe_profile in recipe_profile_list:
                for key in self.attributes:
                    wandb.log({key: recipe_profile[key]})
            
       
        # 평가 지표 계산
        performance = sum(evaluation_results[metric] for metric in self.evaluation_metrics)
                # Best 모델 판정 및 저장
        threshold_performance = 2.07
        threshold_abv_match = 0.656
        threshold_taste_match = 0.616
        
        self.save_best_model(
                            self.model, 
                            performance, 
                            evaluation_results['abv_match'], 
                            evaluation_results['taste_match'], 
                            threshold_performance, 
                            threshold_abv_match, 
                            threshold_taste_match,wandb.run.id) 

        if self.wandb:
            # 평가 지표 로깅
            wandb.log({'performance': performance, **evaluation_results})
            wandb.finish()
        return loss, accuracy, performance





# # # 모델 학습
# # recipe_generation_model.train(train_recipes, epochs=50, batch_size=32)
# def train_with_sweep():
#     # CocktailEmbeddingMaker 인스턴스 생성
#     cocktail_embedding_maker = CocktailEmbeddingMaker(json_data, flavor_data)

#     # RecipeGenerationModel 인스턴스 생성
#     recipe_generation_model = RecipeGenerationModel(cocktail_embedding_maker, True, max_recipe_length=10)

#     # 학습 데이터 준비
#     train_recipes = [recipe['recipe'].keys() for recipe in json_data['cocktail_info']]

#     # 모델 학습
#     loss, accuracy, performance = recipe_generation_model.train(train_recipes)


#     return loss, accuracy, performance



# sweep_configuration = {
#     'method': 'random',
#     'name': 'sweep',
#     'metric': {'goal': 'maximize', 'name': 'performance'},
#     'parameters': 
#     {
#         'batch_size': {'values': [16,32,64]},
#         'epochs': {'values': [200,300]},
#         'lr': {'max': 0.1, 'min': 0.07}
#      }
# }


# sweep_id = wandb.sweep(sweep_configuration, project='cocktail_recipe_generation')
# wandb.agent(sweep_id, function=train_with_sweep)

cocktail_embedding_maker = CocktailEmbeddingMaker(json_data, flavor_data)

# RecipeGenerationModel 인스턴스 생성
recipe_generation_model = RecipeGenerationModel(cocktail_embedding_maker, False, max_recipe_length=10)

# 학습 데이터 준비
train_recipes = [recipe['recipe'].keys() for recipe in json_data['cocktail_info']]

# 모델 학습
loss, accuracy, performance = recipe_generation_model.train(train_recipes)
recipe_generation_model.model.model.save(f'testmodel.h5')

2024-05-30 11:09:53.195709: I external/local_tsl/tsl/cuda/cudart_stub.cc:32] Could not find cuda drivers on your machine, GPU will not be used.
2024-05-30 11:09:53.199523: I external/local_tsl/tsl/cuda/cudart_stub.cc:32] Could not find cuda drivers on your machine, GPU will not be used.
2024-05-30 11:09:53.235695: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-05-30 11:09:53.915023: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT
/usr/lib/python3/dist-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.17.3 and <1.25.0 is required for this version of SciPy (detected version 1.26.4
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"
/home/ysy/.local/lib/python3.10/sit

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 146ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 12ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 12ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 11ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 12ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 11ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 12ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 11ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 13ms/step
s : 0.3913968438320654, d : 0.8028839669421488, a : 0.6096858638743454, t : 0.5435714285714286
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 12ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 13ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 11ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 13ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 13ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 12ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 12ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 12ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 12ms/step
s : 0.46726931351599776, d : 0.802886347107438, a : 0.9857493206071452, t : 0.5218996960486322
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 11ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 12ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 12ms/step
1/1 ━━━━━━━

AttributeError: 'NoneType' object has no attribute 'id'

In [3]:
recipe_generation_model.model.save(f'testmodel.h5')

In [ ]:
wandb.restore('best_model.h5', run_path='wandb/run-20210929_062153-1v5z1z1o/files/best_model.h5')

In [1]:
import wandb

api = wandb.Api()
run = api.run("jennyshin_gist_2024/cocktail_recipe_generation/a9k8jlwi")
best_hyperparams = run.config

# Inference 

In [6]:
from tensorflow import keras
import numpy as np
import tensorflow as tf
import json
from CocktailEmbeddingMaker import Eval
# 데이터 로드
with open('./train_data.json', 'r') as f:
    json_data = json.load(f)
with open('../flavor.json', 'r') as f:
    flavor_data = json.load(f)


attributes = ['ABV', 'boozy', 'sweet', 'sour', 'bitter', 'umami', 'salty', 'astringent', 'Perceived_temperature', 'spicy', 'herbal', 'floral', 'fruity', 'nutty', 'creamy', 'smoky']
#가상의 유저 만들기
test_user_list = []
user_num = 5
for i in range(user_num):
    user = {}
    # user['user_id'] = i
    preference = {}
    user['ABV'] = np.random.randint(0,60)
    for attribute in attributes[1:]:
        user[attribute] = np.random.randint(0,100)
    test_user_list.append(user)

#음료의 총 량
total_amount = 200
#모델 로드
load_model = keras.models.load_model('testmodel.h5')
#Eval 객체 생성 -> inference용 전용 class가 필요하면 이야기할것
Eval_obj = Eval(json_data,flavor_data,load_model)

generated_recipes = Eval_obj.generate_recipe('triple sec',test_user_list[1],3)
result_recipe ={}

for recipe, ingredients in zip(generated_recipes[0], generated_recipes[1]):
    result_recipe[recipe]=ingredients * total_amount
user_recipe_profile = Eval_obj.get_taste_log(generated_recipes)

print(" please ignore user_id!")
print(f"[INPUT] user preference: {json.dumps(test_user_list[1],indent=4)}")
print(f"[OUTPUT]recipe!!!!: {result_recipe} ")
print(f"[OUTPUT]user_recipe's profile : {json.dumps(user_recipe_profile,indent=4)}")


(5,)


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 158ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 12ms/step
 please ignore user_id!
[INPUT] user preference: {
    "ABV": 23,
    "boozy": 99,
    "sweet": 74,
    "sour": 40,
    "bitter": 82,
    "umami": 5,
    "salty": 40,
    "astringent": 30,
    "Perceived_temperature": 42,
    "spicy": 53,
    "herbal": 78,
    "floral": 55,
    "fruity": 95,
    "nutty": 30,
    "creamy": 45,
    "smoky": 96
}
[OUTPUT]recipe!!!!: {'triple sec': 66.66666666666613, 'amaretto': 66.66666666666613, 'cider': 66.66666666666613} 
[OUTPUT]user_recipe's profile : {
    "ABV": 22.333333333333332,
    "boozy": 36.666666666666664,
    "sweet": 53.33333333333333,
    "sour": 6.666666666666666,
    "bitter": 3.333333333333333,
    "umami": 3.333333333333333,
    "salty": 0.0,
    "astringent": 3.333333333333333,
    "Perceived_temperature": 26.666666666666664,
    "spicy": 0.0,
    "herbal": 6.666666666666666,
    "floral": 13.333333333333332,
    "fruity": 43.33333333333333,
    "nutty": 10

[{'vodka': 66.66666666666613},
 {'ouzo': 66.66666666666613},
 {'dry vermouth': 66.66666666666613}]

In [6]:
# Eval_obj.evaluate_model(load_model,test_user_list)
generated_recipes = Eval_obj.generate_recipe('vodka',test_user_list[1],4)
Eval_obj.get_taste_log(generated_recipes)
total_amount = 200
print(generated_recipes)
print([item*total_amount for item in generated_recipes[1]])

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 19ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 17ms/step
(['vodka', 'peach schnapps', 'chambord raspberry liqueur', 'midori melon liqueur'], [0.25, 0.25, 0.25, 0.25])
[50.0, 50.0, 50.0, 50.0]


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 18ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 17ms/step
['vodka', 'ouzo', 'dry vermouth']
[0.33333333333333065, 0.33333333333333065, 0.33333333333333065, 0.33333333333333065, 0.33333333333333065, 0.33333333333333065, 0.33333333333333065, 0.33333333333333065, 0.33333333333333065, 0.33333333333333065, 0.33333333333333065, 0.33333333333333065, 0.33333333333333065, 0.33333333333333065, 0.33333333333333065, 0.33333333333333065, 0.33333333333333065, 0.33333333333333065, 0.33333333333333065, 0.33333333333333065, 0.33333333333333065, 0.33333333333333065, 0.33333333333333065, 0.33333333333333065, 0.33333333333333065, 0.33333333333333065, 0.33333333333333065, 0.33333333333333065, 0.33333333333333065, 0.33333333333333065, 0.33333333333333065, 0.33333333333333065, 0.33333333333333065, 0.33333333333333065, 0.33333333333333065, 0.33333333333333065, 0.33333333333333065, 0.33333333333333065, 0.33333333333333065, 0.33333333333333065, 0.33333333333333065, 0.33333333333333065, 0.33333

[66.66666666666613, 66.66666666666613, 66.66666666666613]